# COVER

In [1]:
import os
import numpy as np
from PIL import Image, ImageDraw
from scipy.ndimage import label, find_objects
from collections import Counter

def find_color_blocks(image, margin=10):
    """
    Identifies all bounding boxes of color blocks in the image.
    Arguments:
    - image: PIL Image object
    - margin: Extra margin around the detected color block
    Returns:
    - List of bounding boxes (x_min, y_min, x_max, y_max)
    """
    img = image.convert("RGB")
    img_data = np.array(img)
    width, height = img.size

    # Define color threshold (greenish range)
    min_r, max_r, min_g, max_g, min_b, max_b = 0, 255, 100, 255, 100, 255
    mask = (
        (img_data[:, :, 0] >= min_r) & (img_data[:, :, 0] <= max_r) &
        (img_data[:, :, 1] >= min_g) & (img_data[:, :, 1] <= max_g) &
        (img_data[:, :, 2] >= min_b) & (img_data[:, :, 2] <= max_b)
    )

    # Find connected components
    from scipy.ndimage import label, find_objects
    labeled, num_features = label(mask)
    slices = find_objects(labeled)

    bounding_boxes = []
    for s in slices:
        x_min, x_max = s[1].start, s[1].stop
        y_min, y_max = s[0].start, s[0].stop
        bounding_boxes.append((x_min, y_min, x_max, y_max))

    return bounding_boxes

def find_closest_to_center(image, bounding_boxes):
    """
    Finds the bounding box closest to the center of the image.
    Arguments:
    - image: PIL Image object
    - bounding_boxes: List of bounding boxes (x_min, y_min, x_max, y_max)
    Returns:
    - Closest bounding box to the center
    """
    width, height = image.size
    center_x, center_y = width // 2, height // 2

    def distance_to_center(box):
        x_min, y_min, x_max, y_max = box
        box_center_x = (x_min + x_max) // 2
        box_center_y = (y_min + y_max) // 2
        return ((box_center_x - center_x) ** 2 + (box_center_y - center_y) ** 2) ** 0.5

    closest_box = min(bounding_boxes, key=distance_to_center)
    return closest_box

def analyze_common_size(bounding_boxes):
    """
    Phân tích kích thước phổ biến nhất của các bounding box.
    Arguments:
    - bounding_boxes: List các bounding box (x_min, y_min, x_max, y_max)
    Returns:
    - Kích thước phổ biến nhất (width, height)
    """
    sizes = [(x_max - x_min, y_max - y_min) for x_min, y_min, x_max, y_max in bounding_boxes]
    size_counts = Counter(sizes)
    most_common_size = size_counts.most_common(1)[0][0]  # Lấy kích thước phổ biến nhất
    return most_common_size

def adjust_to_common_size_with_fixed_right(box, common_size):
    """
    Điều chỉnh bounding box về kích thước phổ biến, giữ nguyên cạnh bên phải.
    Arguments:
    - box: Bounding box hiện tại (x_min, y_min, x_max, y_max)
    - common_size: Kích thước phổ biến (width, height)
    Returns:
    - Bounding box đã điều chỉnh
    """
    x_min, y_min, x_max, y_max = box
    current_width = x_max - x_min
    current_height = y_max - y_min
    target_width, target_height = common_size

    # Điều chỉnh cạnh trái và cạnh trên sao cho kích thước khớp
    new_x_min = max(0, x_max - target_width)  # Giữ nguyên cạnh phải, thay đổi cạnh trái
    new_y_min = max(0, y_min)  # Cạnh trên cố định
    new_y_max = new_y_min + target_height  # Điều chỉnh cạnh dưới

    return new_x_min, new_y_min, x_max, new_y_max

def process_images_with_adjusted_closest_box(input_folder, output_folder, resize_size=(256, 256)):
    """
    Process all images, tìm kích thước phổ biến nhất của closest_box
    và tinh chỉnh bounding box gần trung tâm về kích thước đó.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    all_closest_boxes = []

    # Bước 1: Thu thập tất cả closest_box từ các ảnh
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            input_path = os.path.join(input_folder, filename)
            image = Image.open(input_path)
            bounding_boxes = find_color_blocks(image)
            if bounding_boxes:
                closest_box = find_closest_to_center(image, bounding_boxes)
                all_closest_boxes.append(closest_box)

    # Bước 2: Xác định kích thước phổ biến nhất của closest_box
    if not all_closest_boxes:
        print("Không tìm thấy bounding box nào.")
        return
    common_size = analyze_common_size(all_closest_boxes)
    print(f"Kích thước phổ biến nhất của closest_box: {common_size}")

    # Bước 3: Tinh chỉnh closest_box theo kích thước phổ biến
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            input_path = os.path.join(input_folder, filename)
            image = Image.open(input_path)

            bounding_boxes = find_color_blocks(image)
            if bounding_boxes:
                closest_box = find_closest_to_center(image, bounding_boxes)

                # Điều chỉnh closest_box về kích thước phổ biến
                adjusted_closest_box = adjust_to_common_size_with_fixed_right(closest_box, common_size)

                # Vẽ và lưu ảnh kết quả
                cropped_image = image.crop(adjusted_closest_box)

                # Resize ảnh đã cắt thành 256x256
                resized_image = cropped_image.resize(resize_size)

                # Lưu ảnh kết quả
                output_path = os.path.join(output_folder, filename)
                resized_image.save(output_path)
                print(f"Processed and saved: {output_path}")
            else:
                print(f"No color block found in: {filename}")

# Example usage:
input_folder = './temp/'  # Replace with the path to your input folder
output_folder = './NO2/'  # Replace with the path to your output folder
process_images_with_adjusted_closest_box(input_folder, output_folder)

Kích thước phổ biến nhất của closest_box: (635, 659)
Processed and saved: ./NO2/0sv9g_ngochanpham274@gmail.com_2024-11-29 11_27_21_NO2_0.25ppm_002_samsung test_10.876833_106.6782313.jpg
Processed and saved: ./NO2/15Ukv_ngochanpham274@gmail.com_2024-11-29 11_38_20_NO2_0.9ppm_002_samsung test_10.876833_106.6782313.jpg
Processed and saved: ./NO2/2d7cQ_ngochanpham274@gmail.com_2024-11-29 11_34_05_NO2_0.6ppm_002_samsung test_10.876833_106.6782313.jpg
Processed and saved: ./NO2/2En27_ngochanpham274@gmail.com_2024-11-29 11_56_24_NO2_9ppm_001_samsung test_10.8768544_106.6782314.jpg
Processed and saved: ./NO2/314Lk_ngochanpham274@gmail.com_2024-11-29 11_03_36_NO2_0.025ppm_002_samsung test_10.8768398_106.6782302.jpg
Processed and saved: ./NO2/37F7I_ngochanpham274@gmail.com_2024-11-29 11_28_50_NO2_0.4ppm_001_samsung test_10.876833_106.6782313.jpg
Processed and saved: ./NO2/3p8Kr_ngochanpham274@gmail.com_2024-11-29 11_12_03_NO2_0.06ppm_003_samsung test_10.8768629_106.6782381.jpg
Processed and save

# MIDDLE

In [7]:
def crop_and_resize_center_square(input_folder, output_folder, resize_size=(256, 256)):
    """
    Cắt hình vuông nằm giữa ảnh, có diện tích bằng 1/3 diện tích ảnh gốc,
    sau đó resize thành 256x256 và lưu vào folder khác.
    Arguments:
    - input_folder: Thư mục chứa ảnh đã resize (256x256)
    - output_folder: Thư mục lưu ảnh kết quả
    - resize_size: Kích thước đích (width, height) cho ảnh sau khi resize
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            input_path = os.path.join(input_folder, filename)
            image = Image.open(input_path)
            width, height = image.size

            # Xác định kích thước của hình vuông với diện tích bằng 1/3 diện tích ảnh gốc
            side_length = int((width * height / 3) ** 0.5)

            # Xác định tọa độ để hình vuông nằm giữa ảnh
            center_x, center_y = width // 2, height // 2
            x_min = max(0, center_x - side_length // 2)
            y_min = max(0, center_y - side_length // 2)
            x_max = min(width, x_min + side_length)
            y_max = min(height, y_min + side_length)

            # Cắt hình vuông ở giữa
            cropped_image = image.crop((x_min, y_min, x_max, y_max))

            # Resize hình vuông đã cắt thành 256x256
            resized_image = cropped_image.resize(resize_size)

            # Lưu ảnh kết quả
            output_path = os.path.join(output_folder, filename)
            resized_image.save(output_path)
            print(f"Processed and saved: {output_path}")

# Gọi hàm xử lý
intermediate_folder = './samsung_cover/'  # Thư mục chứa ảnh resize 256x256 ban đầu
final_folder = './samsung_middle/'       # Thư mục lưu kết quả cuối cùng
crop_and_resize_center_square(intermediate_folder, final_folder)


Processed and saved: ./samsung_middle/0iOM6_ngochanpham274@gmail.com_2024-11-21 12_52_30_NH3_5ppm_003_samsung test__.jpg
Processed and saved: ./samsung_middle/1iVDC_ngochanpham274@gmail.com_2024-11-21 12_04_02_NH3_0,25ppm_003_samsung test__.jpg
Processed and saved: ./samsung_middle/2qR82_ngochanpham274@gmail.com_2024-11-21 12_00_28_NH3_0,09ppm_001_samsung test__.jpg
Processed and saved: ./samsung_middle/3iW5i_ngochanpham274@gmail.com_2024-11-21 11_48_30_NH3_0,01ppm_003_samsung test_10.8768228_106.6782789.jpg
Processed and saved: ./samsung_middle/3IwPn_ngochanpham274@gmail.com_2024-11-21 11_59_52_NH3_0,075ppm_003_samsung test__.jpg
Processed and saved: ./samsung_middle/4fkPP_ngochanpham274@gmail.com_2024-11-21 12_50_09_NH3_4ppm_001_samsung test__.jpg
Processed and saved: ./samsung_middle/5RgJx_ngochanpham274@gmail.com_2024-11-21 11_55_08_NH3_0,05ppm_002_samsung test_10.8768228_106.6782789.jpg
Processed and saved: ./samsung_middle/5ufqX_ngochanpham274@gmail.com_2024-11-21 12_47_34_NH3_1p